In [ ]:
yoi = [2020]
toi = ['15RTN', '15RTP', '14RNS']
root_path = "C:/Users/mcecil/CGA/CDL/"
spath = root_path + f"CDL_HLS_dataframe{yoi[0]}.csv"
url_file = root_path + f'CDL_HLS_dataframe{yoi[0]}.csv'
index_file = root_path + f'mask_index_{yoi[0]}.csv'
kml_file = root_path + 'sentinel_tile_grid.kml'
geojson_file = root_path + 'aoi.geojson'  ## chip file, lat-long

geojson_dir = root_path + 'geojson/'

chip_dir = root_path + 'chips/'

image_index_file = root_path + f'image_index_{yoi[0]}.csv'


geojson_rpj_file = root_path + 'aoi_rpj.geojson'
hdf_dir = root_path + 'hdf/'
tiff_dir = root_path + 'tif/'
tiff_dir_rpj = root_path + 'tif_5070/'

mask_dir = root_path + 'mask/'

tile_src_path = root_path + "tile_src.csv"

# New tile edge length (in pixels)
npixels = 512

# Directory containing scenes
scdir = root_path + 'scenes/'

# Directory containing masks
madir = root_path +  'masks/'

# Location to save new files
svdir = root_path +  'subsetted_USSE/'


In [ ]:

# Switch plotting backend
import matplotlib
matplotlib.use('agg')

# Import required modules
#import cartopy.crs as ccrs
import os
from datetime import datetime
import pandas as pd
import geopandas
#import pyhdf
import rasterio
import rasterio.mask
import matplotlib.pyplot as pp
import nasa_hls
import numpy as np
import time
import fiona
from pathlib import Path
import time
import urllib.request as urlreq
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# Switch plotting backend
import matplotlib
matplotlib.use('agg')


### Import modules
from glob import glob
import numpy as np
import matplotlib.pyplot as pp
import rasterio

In [ ]:
from rasterio.windows import Window

### Locate mask files and loop over them
masks = sorted(glob(madir+'*.tif'))
#print(masks)

Comparison of gdal and rasterio crs

In [ ]:
from osgeo import gdal,osr
ds=gdal.Open(r'C:/Users/mcecil/CGA/CDL/tif_5070_new//HLS.S30.T14RNS.2020107.v1.4/HLS.S30.T14RNS.2020107.v1.4__Blue__5070.tif')
prj=ds.GetProjection()
print (prj)
print("")
r = rasterio.open(r'C:/Users/mcecil/CGA/CDL/tif_5070_new//HLS.S30.T14RNS.2020107.v1.4/HLS.S30.T14RNS.2020107.v1.4__Blue__5070.tif')
print(r.crs)

#polygon = polygon.to_crs(raster.crs.to_dict()) 

Set up

In [ ]:
geojson_files = sorted(glob(geojson_dir+'*.geojson'))
print(geojson_files)

In [ ]:
# Load the HLS tiles and place there coordinates into a numpy array for processing later
fiona.drvsupport.supported_drivers['KML'] = 'rw'
tile_src = geopandas.read_file(kml_file, driver='KML')
tile_name = []
tile_x = []
tile_y = []
for tile_ind in range(tile_src.shape[0]):
    tile_name.append(tile_src.iloc[tile_ind].Name)
    tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
    tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
tile_name = np.array(tile_name)
tile_x = np.array(tile_x)
tile_y = np.array(tile_y)
tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
#del tile_src
tile_src.head(5)

Create chips using GDAL crs (works)

In [ ]:
from rasterio.windows import Window
import rasterio
from rasterio.mask import mask as m
import json
import pyproj
  
## load image index 
image_index = pd.read_csv(image_index_file)
#print(image_index)
    
# Scale value for HLS S30 bands
scale = 0.0001

for k in range(len(geojson_files)):
    geojson = geojson_files[k]
    aoi_src = geopandas.read_file(geojson) ## this could be a list
   # aoi_src = aoi_src.to_crs()
    aoi_src = pd.concat([aoi_src, aoi_src.bounds], axis = 1)
    aoi_src['centroid_x'] = aoi_src.centroid.x
    aoi_src['centroid_y'] = aoi_src.centroid.y
    print(aoi_src)
    aoi_row = aoi_src.iloc[0]
   
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
    
    chip_id = str(k)
        
    # Identify closest tile
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
   # print(s)
   # print(tname)


    candidate_images_df = image_index.loc[(image_index['tile'] == tname) & (image_index['convert_to_tif'] == True)]
    candidate_images = candidate_images_df['image_id'].tolist()
    print(candidate_images)
    for image in candidate_images[0:5]:
        print('processing image: ' + image)
        ### Comment out unwanted bands
        #src1 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Coastal_Aerosol__5070.tif')
        print('file: ' + f'{tiff_dir_rpj}/{image}/{image}__Blue__5070.tif')
        
        src2 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Blue__5070.tif')
        src3 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Green__5070.tif')
        src4 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red__5070.tif')
        #src5 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red_Edge1__5070.tif')
        #src6 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red_Edge2__5070.tif')
        #src7 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red_Edge3__5070.tif')
        #src8 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__NIR_Broad__5070.tif')
        src8A = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__NIR_Narrow__5070.tif')
        src11 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__SWIR1__5070.tif')
        src12 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__SWIR2__5070.tif')
        #src9 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Water_Vapor__5070.tif')
        #src10 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Cirrus.__5070tif')
        srcQA = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__QA__5070.tif')
      #  src_template = 
       # print(src_template.crs)
        print('aoi crs')
        print(aoi_src.crs)
        print('raster crs')
        print(src2.crs)
       

        
        ## read in blue band for transformation

        out_image_template, out_transform_template = rasterio.mask.mask(src2, [aoi_shape.geometry], crop=True)
        print(out_image_template.shape)
        with src2 as src:
            meta = src.meta.copy()
            band = np.clip(out_image_template*scale, 0, 1)[0]
            crs_crop = gdal.Open(r'C:/Users/mcecil/CGA/CDL/tif_5070_new//HLS.S30.T14RNS.2020107.v1.4/HLS.S30.T14RNS.2020107.v1.4__Blue__5070.tif').GetProjection()
            print(band.shape)
            
            
        aoi_shape = aoi_src.to_crs(crs_crop).iloc[0]
      #  aoi_shape = aoi_src.iloc[0]

        print(aoi_shape)
        bands = [band]
        
        for src in (src3,src4,src8A,src11,src12, srcQA): # Set the tuple to match desired bands
            out_image, out_transform = rasterio.mask.mask(src, [aoi_shape.geometry], crop=True)
            print(out_image_template.shape)

            # Scale and clip reflectances
            band = np.clip(out_image*scale, 0, 1)[0]
            bands.append(band)
        
        bands = np.array(bands)
        print('bands')
        print(bands)
        print(bands.shape)
        ### Locate missing data and apply to the mask file
      #  missing_data = bands[0] == 0
  #  subset_data[0, missing_data] = -1
        
    # Update the metadata with the new dimensions and transform
        meta.update({"driver": "GTiff",
                 "height": out_image_template.shape[1],
                 "width": out_image_template.shape[2],
                 "count": bands.shape[0],
                 "transform": out_transform_template,
                 "crs": crs_crop   ,
                 "dtype": 'float32'})
        print('meta')
        print(meta)
        save_name = f'{chip_dir}/{image}_chip_{chip_id}.tif'

        with rasterio.open(save_name, 'w', **meta) as dest:
            dest.write(bands)
    
                 
    # meta.update({"count": 6,
    #             'dtype': 'float32'})
    # ### Save the scene tiff
    # out_meta.update({'driver':'GTiff', 'height':bands.shape[1],
    #     'width':bands.shape[2], 'count':bands.shape[0], 'dtype':bands.dtype})
    # save_name = f'{svdir}/scenes/subsetted_{npixels}x{npixels}_{name}_merged_2.tif'
    # print(meta)
    # print(bands.shape)
    # with rasterio.open(save_name, 'w', **meta) as dest:
    #     dest.write(bands)

Crop using rasterio crs (does not work)

In [171]:
from rasterio.windows import Window
import rasterio
from rasterio.mask import mask as m
import json
import pyproj
  
## load image index 
image_index = pd.read_csv(image_index_file)
#print(image_index)
    
# Scale value for HLS S30 bands
scale = 0.0001

for k in range(len(geojson_files)):
    geojson = geojson_files[k]
    aoi_src = geopandas.read_file(geojson) ## this could be a list
   # aoi_src = aoi_src.to_crs()
    aoi_src = pd.concat([aoi_src, aoi_src.bounds], axis = 1)
    aoi_src['centroid_x'] = aoi_src.centroid.x
    aoi_src['centroid_y'] = aoi_src.centroid.y
    print(aoi_src)
    aoi_row = aoi_src.iloc[0]
   
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
    
    chip_id = str(k)
        
    # Identify closest tile
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
   # print(s)
   # print(tname)


    candidate_images_df = image_index.loc[(image_index['tile'] == tname) & (image_index['convert_to_tif'] == True)]
    candidate_images = candidate_images_df['image_id'].tolist()
    print(candidate_images)
    for image in candidate_images[0:5]:
        print('processing image: ' + image)
        ### Comment out unwanted bands
        #src1 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Coastal_Aerosol__5070.tif')
        print('file: ' + f'{tiff_dir_rpj}/{image}/{image}__Blue__5070.tif')
        
        src2 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Blue__5070.tif')
        src3 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Green__5070.tif')
        src4 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red__5070.tif')
        #src5 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red_Edge1__5070.tif')
        #src6 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red_Edge2__5070.tif')
        #src7 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red_Edge3__5070.tif')
        #src8 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__NIR_Broad__5070.tif')
        src8A = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__NIR_Narrow__5070.tif')
        src11 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__SWIR1__5070.tif')
        src12 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__SWIR2__5070.tif')
        #src9 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Water_Vapor__5070.tif')
        #src10 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Cirrus.__5070tif')
        srcQA = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__QA__5070.tif')
      #  src_template = 
       # print(src_template.crs)
        print('aoi crs')
        print(aoi_src.crs)
        print('raster crs')
        print(src2.crs)
       

        
        ## read in blue band for transformation

        out_image_template, out_transform_template = rasterio.mask.mask(src2, [aoi_shape.geometry], crop=True)
        print(out_image_template.shape)
        with src2 as src:
            meta = src.meta.copy()
            band = np.clip(out_image_template*scale, 0, 1)[0]
           # crs_crop = gdal.Open(r'C:/Users/mcecil/CGA/CDL/tif_5070_new//HLS.S30.T14RNS.2020107.v1.4/HLS.S30.T14RNS.2020107.v1.4__Blue__5070.tif').GetProjection()
            print(band.shape)
            crs_crop = src.crs
            
            
        aoi_shape = aoi_src.to_crs(crs_crop).iloc[0]
      #  aoi_shape = aoi_src.iloc[0]

        print(aoi_shape)
        bands = [band]
        
        for src in (src3,src4,src8A,src11,src12, srcQA): # Set the tuple to match desired bands
            out_image, out_transform = rasterio.mask.mask(src, [aoi_shape.geometry], crop=True)
            print(out_image_template.shape)

            # Scale and clip reflectances
            band = np.clip(out_image*scale, 0, 1)[0]
            bands.append(band)
        
        bands = np.array(bands)
        print('bands')
        print(bands)
        print(bands.shape)
        ### Locate missing data and apply to the mask file
      #  missing_data = bands[0] == 0
  #  subset_data[0, missing_data] = -1
        
    # Update the metadata with the new dimensions and transform
        meta.update({"driver": "GTiff",
                 "height": out_image_template.shape[1],
                 "width": out_image_template.shape[2],
                 "count": bands.shape[0],
                 "transform": out_transform_template,
                 "crs": crs_crop   ,
                 "dtype": 'float32'})
        print('meta')
        print(meta)
        save_name = f'{chip_dir}/{image}_chip_{chip_id}.tif'

        with rasterio.open(save_name, 'w', **meta) as dest:
            dest.write(bands)
    
                 
    # meta.update({"count": 6,
    #             'dtype': 'float32'})
    # ### Save the scene tiff
    # out_meta.update({'driver':'GTiff', 'height':bands.shape[1],
    #     'width':bands.shape[2], 'count':bands.shape[0], 'dtype':bands.dtype})
    # save_name = f'{svdir}/scenes/subsetted_{npixels}x{npixels}_{name}_merged_2.tif'
    # print(meta)
    # print(bands.shape)
    # with rasterio.open(save_name, 'w', **meta) as dest:
    #     dest.write(bands)

C:\Users\mcecil\AppData\Local\Temp\ipykernel_10040\3321074966.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_src['centroid_x'] = aoi_src.centroid.x
C:\Users\mcecil\AppData\Local\Temp\ipykernel_10040\3321074966.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_src['centroid_y'] = aoi_src.centroid.y


                                            geometry       minx       miny   
0  POLYGON ((-98.46590 28.48216, -98.46590 28.491... -98.492844  28.482162  \

        maxx       maxy  centroid_x  centroid_y  
0 -98.465904  28.491892  -98.479374   28.487027  
['HLS.S30.T14RNS.2020107.v1.4', 'HLS.S30.T14RNS.2020110.v1.4', 'HLS.S30.T14RNS.2020115.v1.4', 'HLS.S30.T14RNS.2020117.v1.4', 'HLS.S30.T14RNS.2020122.v1.4', 'HLS.S30.T14RNS.2020127.v1.4', 'HLS.S30.T14RNS.2020130.v1.4', 'HLS.S30.T14RNS.2020135.v1.4', 'HLS.S30.T14RNS.2020140.v1.4', 'HLS.S30.T14RNS.2020145.v1.4', 'HLS.S30.T14RNS.2020147.v1.4', 'HLS.S30.T14RNS.2020155.v1.4', 'HLS.S30.T14RNS.2020157.v1.4', 'HLS.S30.T14RNS.2020160.v1.4', 'HLS.S30.T14RNS.2020165.v1.4', 'HLS.S30.T14RNS.2020167.v1.4', 'HLS.S30.T14RNS.2020172.v1.4', 'HLS.S30.T14RNS.2020185.v1.4', 'HLS.S30.T14RNS.2020187.v1.4', 'HLS.S30.T14RNS.2020190.v1.4', 'HLS.S30.T14RNS.2020192.v1.4', 'HLS.S30.T14RNS.2020195.v1.4', 'HLS.S30.T14RNS.2020197.v1.4', 'HLS.S30.T14RNS.2020200.v1.4'

ProjError: Error creating Transformer from CRS.